# KRW‑USDT 5년 백테스트 📈

이 노트북은 **그리드 트레이딩 전략**과 **Buy & Hold**를 비교하기 위한 예시입니다.  
- 5 년치 KRW‑USDT 일봉 데이터를 받아서  
- 박스폭(스플릿 구간)을 여러 값으로 바꿔가며 그리드 성과를 측정하고  
- 수익률·MDD·거래횟수·승률을 표로 정리합니다.

> ⚠️ 실제 투자 전 반드시 매커니즘을 이해하고, 수수료·슬리피지를 현실 값으로 조정해 보세요.


## 1. 라이브러리 설치

아래 셀을 실행해 필요한 라이브러리를 설치하세요 (Colab / Jupyter).

```bash
pip install pyupbit pandas numpy matplotlib tqdm
```

In [ ]:
# 2. 공통 import 및 기본 설정
import pandas as pd, numpy as np, pyupbit, datetime as dt
from tqdm import tqdm

FEE_RATE = 0.0005          # 업비트 기준 수수료 0.05 %
STEP_WON = 4               # 그리드 간격 (원)
MAX_GRID  = 10             # 한쪽 방향 최대 계단 수
ORDER_KRW = 5_000          # 각 계단당 주문 금액 (원)


In [ ]:
# 3. 5년치 KRW‑USDT 일봉 데이터 다운로드
def fetch_5y_ohlcv(market="KRW-USDT"):
    end = dt.datetime.utcnow()
    start = end - dt.timedelta(days=365*5 + 10)
    dfs = []
    while start < end:
        to = end.strftime("%Y-%m-%d")
        df  = pyupbit.get_ohlcv(market, interval="day", count=200, to=to)
        if df is None or df.empty:
            break
        dfs.append(df)
        end = df.index[0] - pd.Timedelta(days=1)
    raw = pd.concat(dfs).sort_index().loc[str(start.date()):]
    raw = raw[~raw.index.duplicated()]
    return raw

ohlcv = fetch_5y_ohlcv()
ohlcv.head()

In [ ]:
# 4. Buy & Hold 성과 계산
bh_entry_price = ohlcv['open'].iloc[0]
bh_exit_price  = ohlcv['close'].iloc[-1]
bh_return      = (bh_exit_price / bh_entry_price - 1) * 100

cum_max  = ohlcv['close'].cummax()
drawdown = (ohlcv['close'] / cum_max - 1)
bh_mdd   = drawdown.min() * 100
print(f"Buy & Hold 5Y Return: {bh_return:.2f}%  |  MDD: {bh_mdd:.2f}%")

In [ ]:
# 5. 그리드 시뮬레이터 클래스
class GridSim:
    def __init__(self, base_price, box_low, box_high,
                 step=STEP_WON, max_grid=MAX_GRID):
        self.base = base_price
        self.low  = box_low
        self.high = box_high
        self.step = step
        self.max  = max_grid
        # 상태
        self.krw  = 1_000_000
        self.coin = 0
        self.trades = []   # (type, price, vol)
    # -------------------------------
    def trade_one(self, price):
        # 매수
        while price <= self.base - self.step and self.base - self.step >= self.low               and len([t for t in self.trades if t[0]=='buy']) < self.max:
            self.base -= self.step
            vol = ORDER_KRW / price
            fee = ORDER_KRW * FEE_RATE
            if self.krw < ORDER_KRW + fee:
                break
            self.krw -= ORDER_KRW + fee
            self.coin += vol
            self.trades.append(('buy', price, vol))
        # 매도
        while price >= self.base + self.step and self.base + self.step <= self.high               and self.coin > 0:
            self.base += self.step
            vol = ORDER_KRW / self.base
            vol = min(vol, self.coin)
            proceeds = vol * price
            fee = proceeds * FEE_RATE
            self.krw += proceeds - fee
            self.coin -= vol
            self.trades.append(('sell', price, vol))
    # -------------------------------
    def run(self, prices: np.ndarray):
        for p in prices:
            self.trade_one(p)
        final_assets = self.krw + self.coin * prices[-1]
        ret = (final_assets / 1_000_000 - 1) * 100
        mdd = self._calc_mdd(prices)
        win = self._win_rate()
        return ret, mdd, len(self.trades)//2, win
    # -------------------------------
    def _calc_mdd(self, closes):
        equity = []
        krw, coin = self.krw, self.coin
        for c in closes:
            equity.append(krw + coin * c)
        s = pd.Series(equity)
        return (s / s.cummax() - 1).min() * 100
    def _win_rate(self):
        pairs = list(zip(self.trades[::2], self.trades[1::2]))
        wins = [sell[1] > buy[1] for buy, sell in pairs].count(True)
        return wins / len(pairs)*100 if pairs else 0

In [ ]:
# 6. 박스폭 스윕
results = []
close_prices = ohlcv['close'].values
mid_price = close_prices.mean()
box_sizes = range(40, 201, 20)  # 40원 ~ 200원, 20원 간격
for box in tqdm(box_sizes):
    low  = mid_price - box/2
    high = mid_price + box/2
    gs = GridSim(base_price=mid_price, box_low=low, box_high=high)
    ret, mdd, ntrade, win = gs.run(close_prices)
    results.append([box, ret, mdd, ntrade, win])

df = pd.DataFrame(results, columns=['Box(원)', 'Grid Return %', 'MDD %', 'Trades', 'Win Rate %'])
df.style.format({'Grid Return %':'{:.2f}', 'MDD %':'{:.2f}', 'Win Rate %':'{:.1f}'})

In [ ]:
# 7. 요약 비교
best = df.loc[df['Grid Return %'].idxmax()]
print(f"▶ Buy & Hold 5년 수익률 : {bh_return:.2f}%  /  MDD {bh_mdd:.2f}%")
print('▶ 그리드 최고 성능 :')
display(best)
